In [1]:
#imported required libraries
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score

In [2]:
data=pd.read_csv('https://raw.githubusercontent.com/akshaytheau/Data-Science/master/cuisine_data.csv')

In [3]:
data.head()

,cuisine_description,cuisine
0,romaine lettuce black olives grape tomatoes ga...,greek
1,plain flour ground pepper salt tomatoes ground...,southern_us
2,eggs pepper salt mayonaise cooking oil green c...,filipino
3,water vegetable oil wheat salt,indian
4,black pepper shallots cornflour cayenne pepper...,indian


In [5]:
len(data.cuisine.value_counts())

20

In [6]:
data.cuisine.unique()

array(['greek', 'southern_us', 'filipino', 'indian', 'jamaican',
       'spanish', 'italian', 'mexican', 'chinese', 'british', 'thai',
       'vietnamese', 'cajun_creole', 'brazilian', 'french', 'japanese',
       'irish', 'korean', 'moroccan', 'russian'], dtype=object)

In [7]:
data.isnull().sum() #data preprocessing

cuisine_description    0
cuisine                0
dtype: int64

In [9]:
data.shape

(39774, 2)

In [12]:
data.drop_duplicates(inplace=True)

In [14]:
data.shape

(39677, 2)

In [17]:
print(data['cuisine_description'].apply(lambda x: len(x.split(' '))).sum())

806112


In [18]:
special_character_remover=re.compile('[/(){}\[\]\|@,;]')

In [19]:
extra_symbol_remover=re.compile('[^0.9a-z #+_]')

In [21]:
nltk.download('stopwords')
STOPWORDS= set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
#created a fuction which iterate through whole input data and preprocess the data
def clean_text(text):
    text=text.lower()
    text=special_character_remover.sub(' ',text)
    text=extra_symbol_remover.sub('',text)
    text=' '.join(word for word in text.split() if word not in STOPWORDS)
    return text
data['cuisine_description']=data['cuisine_description'].apply(clean_text)

In [24]:
print(data['cuisine_description'].apply(lambda x: len(x.split(' '))).sum())

802995


In [25]:
#define the inputs to the model x which is independent variable and y as a dependent variable
x= data.cuisine_description
y= data.cuisine


In [26]:
#to convert input data into array
cv=CountVectorizer()
X=cv.fit_transform(x).toarray()

In [27]:
cv=CountVectorizer()
Y=cv.fit_transform(y).toarray()

In [29]:
#to train the model used sklearn train test split method which split the input data into training and testing set
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3, random_state=100)

In [32]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [34]:
X_train.shape, X_test.shape,y_train.shape,y_test.shape

((27773, 3038), (11904, 3038), (27773, 20), (11904, 20))

In [36]:
#used keras to run the model which consist neural network
from keras.models import Sequential
from keras.layers import Dense

In [37]:
model = Sequential()
model.add(Dense(120, input_dim=3038, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(20, activation='sigmoid'))

In [39]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
model.fit(X_train,y_train, epochs=10, batch_size=10)

Epoch 1/10
2778/2778 [==============================] - 50s 7ms/step - loss: 0.1869 - accuracy: 0.4153
Epoch 2/10
2778/2778 [==============================] - 18s 6ms/step - loss: 0.0793 - accuracy: 0.7043
Epoch 3/10
2778/2778 [==============================] - 16s 6ms/step - loss: 0.0659 - accuracy: 0.7637
Epoch 4/10
2778/2778 [==============================] - 16s 6ms/step - loss: 0.0551 - accuracy: 0.8120
Epoch 5/10
2778/2778 [==============================] - 17s 6ms/step - loss: 0.0493 - accuracy: 0.8333
Epoch 6/10
2778/2778 [==============================] - 17s 6ms/step - loss: 0.0431 - accuracy: 0.8641
Epoch 7/10
2778/2778 [==============================] - 17s 6ms/step - loss: 0.0378 - accuracy: 0.8847
Epoch 8/10
2778/2778 [==============================] - 18s 6ms/step - loss: 0.0326 - accuracy: 0.9023
Epoch 9/10
2778/2778 [==============================] - 18s 7ms/step - loss: 0.0292 - accuracy: 0.9175
Epoch 10/10
2778/2778 [==============================] - 18s 7ms/step - l

In [41]:
#evaluate the model
_, accuracy = model.evaluate(X, Y)
print('Accuracy: %.2f' % (accuracy*100))

1240/1240 [==============================] - 10s 7ms/step - loss: 0.0414 - accuracy: 0.8926
Accuracy: 89.26


In [42]:
print(_)

0.04136825352907181


In [43]:
y_pred=model.predict(X)

In [44]:
y_pred=(y_pred>0.5)

In [45]:
y_pred[2]

array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [46]:
Y[2]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)